# Sztuczna inteligencja i inynieria wiedzy - lista 1

In [1]:
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import os

sys.path.append('..')  

from src.data_structures import TransportConnection, TransportStop, TransportRoute
from src.utils import time_to_minutes, minutes_to_time, calculate_distance, estimate_travel_time
from src.data_loader import load_transport_data
from src.visualization import format_route, visualize_route, format_compact_route

from src.algorithms.dijkstra import dijkstra_shortest_path
from src.algorithms.astar import astar_shortest_path, zero_heuristic, distance_heuristic, transfer_penalty_heuristic, direct_line_preference_heuristic, combined_heuristic
from src.algorithms.astar_transfers import astar_min_transfers
from src.algorithms.astar_advanced import astar_bi_criteria, get_cached_transfers_heuristic, get_cached_time_heuristic

from src.tabu_search.tsp_solution import TSPSolution
from src.tabu_search.tabu_search import TabuSearchTSP, determine_tabu_size
from src.data_structures import TransportRoute

## Struktura projektu

### Główne katalogi

- `dane/`: Zawiera pliki z danymi, w tym kluczowy plik connection_graph.csv z rozkładem jazdy komunikacji miejskiej.
- `src/`: Główny katalog zawierający kod źródłowy projektu.

### Moduły w katalogu `src/`

- `algorithms/`: Implementacje algorytmów przeszukiwania:

  - `dijkstra.py`: Implementacja algorytmu Dijkstry
  - `astar.py`: Podstawowa implementacja algorytmu A*
  - `astar_transfers.py`: Wersja A* minimalizująca liczbę przesiadek
  - `astar_advanced.py`: Zaawansowana wersja algorytmu A*


- `tabu_search/`: Moduły związane z przeszukiwaniem Tabu:

  - `tabu_search.py`: Główna implementacja algorytmu przeszukiwania Tabu
  - `tsp_solution.py`: Rozwiązanie problemu komiwojażera


- `utils.py`: Narzędzia pomocnicze, w tym konwersja czasu
- `data_loader.py`: Ładowanie i przetwarzanie danych
- `data_structures.py`: Definiuje struktury danych projektu
- `visualization.py`: Moduł do generowania wizualizacji

### Struktury danych

Projekt wykorzystuje trzy kluczowe klasy:

- **TransportConnection**: Reprezentuje pojedyncze połączenie transportowe
- **TransportStop**: Przechowuje informacje o przystanku i jego połączeniach
- **TransportRoute**: Modeluje kompletną trasę podróży

Taka modułowa struktura projektu pozwala na łatwą rozbudowę i testowanie poszczególnych komponentów systemu.

## Przetwarzanie danych

Proces wczytywania danych obejmuje eliminację duplikatów przystanków, konwersję czasów i współrzędnych oraz generowanie dodatkowych połączeń oczekiwania. Metoda zapewnia spójność danych i przygotowanie kompleksowej struktury grafu połączeń transportowych.

In [2]:
stops, connections = load_transport_data('../dane/connection_graph.csv')

/Users/m_juchiewicz/Desktop/studia/semestr-6/AI/lab01/rozwiazanie/notebooks/../src/data_loader.py:39: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Wczytano 996520 połączeń komunikacyjnych
Utworzono 939 przystanków


## Przykładowe dane do zaprezentowania działania algorytmów

In [3]:
start_stop = "DWORZEC NADODRZE"
end_stop = "Jagodzińska"
start_time = "10:45:00"


# Implementacja algorytmów do wyszukiwania najkrótszych połączeń

## 1.1 Algorytm Dijkstry 

### Zasada działania

Algorytm Dijkstry to klasyczny algorytm wyszukiwania najkrótszej ścieżki w grafie ważonym. Działa na zasadzie "zachłannej" - w każdym kroku wybiera węzeł o najmniejszej dotychczasowej odległości od źródła.

### Implementacja dla transportu publicznego
- **Stan węzła**: Reprezentowany przez parę (przystanek, czas_przyjazdu)
- **Funkcja kosztu:** Czas podróży między przystankami
- **Kolejka priorytetowa:** Uporządkowana według dotychczasowego czasu podróży

### Heurystyka

Dijkstra nie używa żadnej heurystyki - jest to algorytm dokładny, który gwarantuje znalezienie najkrótszej ścieżki pod względem czasu. Można go postrzegać jako szczególny przypadek A* z heurystyką zerową.

### Zaimplementowane optymalizacje
- **Best Arrival Time:** Śledzenie najlepszego czasu przybycia do każdego przystanku
- **Early Termination:** Wcześniejsze zakończenie gdy znajdziemy cel
- **Pruning**: Odcinanie ścieżek, które nie mogą prowadzić do lepszego rozwiązania

Algorytm znajduje się w pliku `algorithms/dijkstra.py`.

### Przykładowe użycie Dijkstry

In [4]:
route = dijkstra_shortest_path(stops, start_stop, end_stop, start_time)
print(format_compact_route(route))

🚉 SZCZEGÓŁY TRASY:
⏱️ Całkowity czas podróży: 47.0 min
🔄 Liczba przesiadek: 3

+-------+------------------+------------------+----------+----------+
| Linia | Skąd             | Dokąd            | Odjazd   | Przyjazd |
+-------+------------------+------------------+----------+----------+
| 7     | DWORZEC NADODRZE | Arkady (Capitol) | 10:45:00 | 11:01:00 |
| 22    | Arkady (Capitol) | Bardzka          | 11:04:00 | 11:17:00 |
| 133   | Bardzka          | Buforowa-Rondo   | 11:20:00 | 11:25:00 |
| 145   | Buforowa-Rondo   | Jagodzińska      | 11:29:00 | 11:32:00 |
+-------+------------------+------------------+----------+----------+


## Wizualizacja trasy

In [5]:
map_viz = visualize_route(route, stops, "Trasa - Dijkstra")
display(map_viz)
print(format_route(route))


📋 HARMONOGRAM PODRÓŻY:

🚍 Linia 7: DWORZEC NADODRZE → Arkady (Capitol)
   Odjazd: 10:45:00, przyjazd: 11:01:00
   Przystanki pośrednie:
     • DWORZEC NADODRZE (10:45:00)
     • Paulińska (10:47:00)
     • Paulińska (10:47:00) [końcowy]
     • Dubois (10:49:00)
     • Dubois (10:49:00) [końcowy]
     • Uniwersytet Wrocławski (10:51:00)
     • Uniwersytet Wrocławski (10:51:00) [końcowy]
     • Rynek (10:54:00)
     • Rynek (10:54:00) [końcowy]
     • Narodowe Forum Muzyki (10:56:00)
     • Narodowe Forum Muzyki (10:56:00) [końcowy]
     • Renoma (10:59:00)
     • Renoma (10:59:00) [końcowy]

🔄 Przesiadka: Arkady (Capitol)
   Czas oczekiwania: 3.0 min (11:01:00 → 11:04:00)

🚍 Linia 22: Arkady (Capitol) → Bardzka
   Odjazd: 11:04:00, przyjazd: 11:17:00
   Przystanki pośrednie:
     • Arkady (Capitol) (11:04:00)
     • DWORZEC GŁÓWNY (11:07:00)
     • DWORZEC GŁÓWNY (11:07:00) [końcowy]
     • Pułaskiego (11:09:00)
     • Pułaskiego (11:09:00) [końcowy]
     • Hubska (Dawida) (11:12:00)
  

## 1.2 Algorytm A* (A-star)

### Zasada działania

A* łączy zalety algorytmu Dijkstry (optymalność) z heurystyką, która kieruje przeszukiwanie w stronę celu, co przyspiesza znalezienie rozwiązania.

**Implementacja dla transportu publicznego**
- Stan węzła: Para (przystanek, czas_przyjazdu)
- Funkcja kosztu: *f(n) = g(n) + h(n)*, gdzie:
    - *g(n)*: Faktyczny czas podróży od początku do węzła n
    - *h(n)*: Szacowany czas podróży od węzła n do celu

### Zaimplementowane heurystyki

- **Zero Heuristic:** Przekształca A* w Dijkstrę
- **Distance Heuristic**: Oblicza odległość euklidesową między przystankami i szacuje czas podróży przy założeniu średniej prędkości 40 km/h
- **Transfer Penalty Heuristic:** Rozszerza heurystykę odległości o kary za przesiadki
- **Direct Line Preference Heuristic:** Preferuje trasy bezpośrednie, sprawdzając czy istnieje bezpośrednie połączenie do celu
- **Combined Heuristic:** Łączy odległość geograficzną, kary za przesiadki i kary za ruch w złym kierunku

### Zaimplementowane optymalizacje
- **Heuristic Caching:** Zapisywanie wyników obliczeń heurystycznych
- **Best Arrival Time Tracking:** Śledzenie najlepszego czasu dotarcia do przystanku
- **Early Termination:** Wcześniejsze zakończenie gdy znajdziemy cel
- **Monotonic Counter:** Licznik do stabilizacji kolejki priorytetowej

Algorytm znajduje się w pliku `algorithms/astar.py`.

### Przykładowe użycie A* z heurystyką odległości

In [6]:
route = astar_shortest_path(stops, start_stop, end_stop, start_time, distance_heuristic)
print(format_compact_route(route))

🚉 SZCZEGÓŁY TRASY:
⏱️ Całkowity czas podróży: 47.0 min
🔄 Liczba przesiadek: 3

+-------+------------------+------------------+----------+----------+
| Linia | Skąd             | Dokąd            | Odjazd   | Przyjazd |
+-------+------------------+------------------+----------+----------+
| 7     | DWORZEC NADODRZE | Arkady (Capitol) | 10:45:00 | 11:01:00 |
| 22    | Arkady (Capitol) | Bardzka          | 11:04:00 | 11:17:00 |
| 133   | Bardzka          | Buforowa-Rondo   | 11:20:00 | 11:25:00 |
| 145   | Buforowa-Rondo   | Jagodzińska      | 11:29:00 | 11:32:00 |
+-------+------------------+------------------+----------+----------+


### Przykładowe użycie A* z heurystyką kary za przesiadki

In [7]:
route = astar_shortest_path(stops, start_stop, end_stop, start_time, transfer_penalty_heuristic)
print(format_compact_route(route))

🚉 SZCZEGÓŁY TRASY:
⏱️ Całkowity czas podróży: 47.0 min
🔄 Liczba przesiadek: 3

+-------+------------------+------------------+----------+----------+
| Linia | Skąd             | Dokąd            | Odjazd   | Przyjazd |
+-------+------------------+------------------+----------+----------+
| 7     | DWORZEC NADODRZE | Arkady (Capitol) | 10:45:00 | 11:01:00 |
| 22    | Arkady (Capitol) | Bardzka          | 11:04:00 | 11:17:00 |
| 133   | Bardzka          | Buforowa-Rondo   | 11:20:00 | 11:25:00 |
| 145   | Buforowa-Rondo   | Jagodzińska      | 11:29:00 | 11:32:00 |
+-------+------------------+------------------+----------+----------+


### Przykładowe użycie A* z heurystyką złozoną

In [8]:
route = astar_shortest_path(stops, start_stop, end_stop, start_time, combined_heuristic)
print(format_compact_route(route))

🚉 SZCZEGÓŁY TRASY:
⏱️ Całkowity czas podróży: 47.0 min
🔄 Liczba przesiadek: 3

+-------+------------------+------------------+----------+----------+
| Linia | Skąd             | Dokąd            | Odjazd   | Przyjazd |
+-------+------------------+------------------+----------+----------+
| 7     | DWORZEC NADODRZE | Arkady (Capitol) | 10:45:00 | 11:01:00 |
| 22    | Arkady (Capitol) | Bardzka          | 11:04:00 | 11:17:00 |
| 133   | Bardzka          | Buforowa-Rondo   | 11:20:00 | 11:25:00 |
| 145   | Buforowa-Rondo   | Jagodzińska      | 11:29:00 | 11:32:00 |
+-------+------------------+------------------+----------+----------+


## 1.3 A* Min Transfers (A* z minimalizacją przesiadek)

### Zasada działania

Modyfikacja A*, która priorytetyzuje trasy z minimalną liczbą przesiadek zamiast (lub obok) minimalnego czasu podróży.

### Implementacja
- **Stan węzła:** Reprezentowany przez parę (przystanek, linia)
- **Funkcja kosztu:** Priorytetyzuje liczbę przesiadek, ignorując czas podróży
- **Kolejka priorytetowa:** Uporządkowana wyłącznie według liczby przesiadek, czas jest drugorzędny

### Zaimplementowane heurystyki

W implementacji A* Min Transfers nie używam złożonych heurystyk. Zamiast tego:

- Priorytetyzuje bezpośrednio liczbę przesiadek w kolejce priorytetowej
- Używa struktury closed set do śledzenia przetworzonych węzłów (przystanek, linia)
- Specjalne traktowanie połączeń na tej samej linii - są sprawdzane jako pierwsze, aby minimalizować przesiadki

### Zaimplementowane optymalizacje

- **Closed Set:** Efektywne śledzenie przetworzonych kombinacji (przystanek, linia)
- **Same Line Priority:** Priorytetyzacja połączeń na tej samej linii przed przesiadkami
- **Null Initial Line:** Rozpoczynanie od stanu z linią ustawioną na null, co pomaga w modelowaniu początku podróży
- **Best Path Search:** Przeszukiwanie wszystkich możliwych tras do celu i wybór tej z minimalną liczbą przesiadek


Algorytm znajduje się w pliku `algorithms/astar_transfers.py`.


### Przykładowe użycie algorytmu A* Min Transfers

In [9]:
route = astar_min_transfers(stops, start_stop, end_stop, start_time)
print(format_compact_route(route))

Szukam trasy z 'DWORZEC NADODRZE' do 'Jagodzińska' od godziny 10:45:00 z minimalną liczbą przesiadek
🚉 SZCZEGÓŁY TRASY:
⏱️ Całkowity czas podróży: 41.0 min
🔄 Liczba przesiadek: 1

+-------+------------------+-------------+----------+----------+
| Linia | Skąd             | Dokąd       | Odjazd   | Przyjazd |
+-------+------------------+-------------+----------+----------+
| 16    | DWORZEC NADODRZE | Bardzka     | 10:56:00 | 11:25:00 |
| 110   | Bardzka          | Jagodzińska | 11:29:00 | 11:37:00 |
+-------+------------------+-------------+----------+----------+


## 1.4 A* Advanced (A* wielokryteriowy)

### Zasada działania

Zaawansowana wersja A*, która uwzględnia równolegle dwa kryteria: czas podróży i liczbę przesiadek, z możliwością przypisania wag.

### Implementacja

- **Stan węzła:** Reprezentowany przez parę (przystanek, linia)
- **Funkcja kosztu:** Kombinowana funkcja łącząca ważony czas podróży i liczbę przesiadek
- **Kolejka priorytetowa**: Uporządkowana według kombinowanej wartości f

### Zaimplementowane heurystyki

- **Hierarchical Time Heuristic:** Zaawansowana heurystyka czasowa, która:
    - Oblicza odległość geograficzną używając wzoru haversine
    - Uwzględnia bezpośrednie połączenia (mnożnik 0.8)
    - Uwzględnia gęstość połączeń (więcej połączeń = szybsza podróż)
    - Używa współczynnika skalowania 0.95 dla zachowania dopuszczalności

- **Min Transfers Heuristic:** Heurystyka szacująca minimalną liczbę przesiadek do celu:
    - 0: Jeśli cel jest bieżącym przystankiem lub istnieje bezpośrednie połączenie
    - 1: Jeśli istnieje wspólna linia między bieżącym przystankiem a celem
    - 2: W pozostałych przypadkach

### Zaimplementowane optymalizacje

- **Heuristic Caching:** Cache dla obliczeń heurystycznych
- **Coordinates Caching:** Cache dla współrzędnych przystanków
- **Connection Caching:** Cache dla połączeń
- **Best Goal Tracking:** Śledzenie najlepszego znalezionego rozwiązania
- **Pruning:** Odcinanie ścieżek, które nie mogą poprawić najlepszego rozwiązania
- **Crossing Midnight Fix:** Korekta dla tras przechodzących przez północ (00:00)

Algorytm znajduje się w pliku `algorithms/astar_advanced.py`.


### Przykladowe użycie ulepszonego A* z hierarchiczną heurystyką czasu podrózy 

In [10]:
route = astar_bi_criteria(stops, start_stop, end_stop, start_time)
print(format_compact_route(route))

🚉 SZCZEGÓŁY TRASY:
⏱️ Całkowity czas podróży: 66.0 min
🔄 Liczba przesiadek: 3

+-------+--------------------+--------------------+----------+----------+
| Linia | Skąd               | Dokąd              | Odjazd   | Przyjazd |
+-------+--------------------+--------------------+----------+----------+
| 714   | DWORZEC NADODRZE   | Kleczkowska        | 10:52:00 | 10:55:00 |
| 142   | Kleczkowska        | Rynek              | 11:01:00 | 11:10:00 |
| K     | Rynek              | DWORZEC AUTOBUSOWY | 11:23:00 | 11:35:00 |
| 110   | DWORZEC AUTOBUSOWY | Jagodzińska        | 11:42:00 | 11:58:00 |
+-------+--------------------+--------------------+----------+----------+


### Przykładowe użycie ulepszonego A* z dodatkowymi parametrami

In [11]:
route = astar_bi_criteria(
    stops, 
    start_stop, 
    end_stop, 
    start_time, 
    transfer_weight=0.7,  
    time_weight=0.3,      
    transfer_time=3
)
print(format_compact_route(route))

🚉 SZCZEGÓŁY TRASY:
⏱️ Całkowity czas podróży: 66.0 min
🔄 Liczba przesiadek: 3

+-------+--------------------+--------------------+----------+----------+
| Linia | Skąd               | Dokąd              | Odjazd   | Przyjazd |
+-------+--------------------+--------------------+----------+----------+
| 714   | DWORZEC NADODRZE   | Kleczkowska        | 10:52:00 | 10:55:00 |
| 142   | Kleczkowska        | Rynek              | 11:01:00 | 11:10:00 |
| K     | Rynek              | DWORZEC AUTOBUSOWY | 11:23:00 | 11:35:00 |
| 110   | DWORZEC AUTOBUSOWY | Jagodzińska        | 11:42:00 | 11:58:00 |
+-------+--------------------+--------------------+----------+----------+


# Porównanie algorytmów

Przeprowadzono analizę porównawczą czterech algorytmów trasowania (Dijkstra, A*, A* Min Transfers, A* Advanced) w oparciu o 30 losowo wybranych połączeń komunikacyjnych. Badanie koncentrowało się na dwóch kluczowych parametrach: **czasie podróży** oraz **liczbie przesiadek**.

| ![Pierwszy obraz](image.png) | ![drugi obraz](transfers_comparison.png) |
|:---:|:---:|

### Wykres 1 (Czas podróży):

Algorytm A* Min Transfers cechuje się najwyższą medianą czasu podróży ze wszystkich porównywanych algorytmów. Na wykresie widoczne są również wartości odstające, szczególnie dla algorytmu A* Min Transfers, wskazujące na dużą zmienność czasu przejazdu.

### Wykres 2 (Liczba przesiadek):

Algorytmy Dijkstra i A* wykazują podobną liczbę przesiadek, podczas gdy A* Min Transfers charakteryzuje się zdecydowanie mniejszą liczbą przesiadek. A* Advanced prezentuje wyższą medianę liczby przesiadek w porównaniu do pozostałych algorytmów.



# Algorytm Tabu Search

## Tabu Search dla problemu TSP w komunikacji miejskiej
Algorytm Tabu Search (TS) jest zaawansowaną metodą optymalizacji kombinatorycznej, która pozwala na efektywne przeszukiwanie przestrzeni rozwiązań problemu. W kontekście zadania znalezienia optymalnej trasy przechodzącej przez wszystkie zadane przystanki w sieci komunikacji miejskiej, TS jest szczególnie odpowiedni ze względu na zdolność do przezwyciężania optymów lokalnych.

## Podstawowe założenia algorytmu Tabu Search

1. **Rozwiązanie początkowe:** Algorytm rozpoczyna działanie od pewnego rozwiązania początkowego, które reprezentuje dopuszczalną trasę rozpoczynającą i kończącą się na przystanku A i przechodząca przez wszystkie przystanki z listy L.

2. **Sąsiedztwo rozwiązania:** W każdej iteracji algorytmu generowane jest sąsiedztwo bieżącego rozwiązania poprzez zastosowanie operatorów ruchu (w tym przypadku zamiana par przystanków).

3. **Lista Tabu:** Kluczowym elementem algorytmu jest lista Tabu, która przechowuje niedawno wykonane ruchy, aby zapobiec cyklicznemu powracaniu do tych samych rozwiązań i umożliwić eksplorację nowych obszarów przestrzeni rozwiązań.

4. **Funkcja celu:** W zależności od wybranego kryterium, funkcja celu może minimalizować czas przejazdu lub liczbę przesiadek (ze sztrafą za czas, aby zapewnić unikalność rozwiązań).

5. **Kryterium aspiracji:** Mechanizm pozwalający na wykonanie ruchu z listy tabu, jeśli spełnia określone kryterium (np. prowadzi do lepszego rozwiązania niż dotychczas znalezione).

## Przykładowe użycie Tabu Search

In [15]:
# Zdefiniuj parametry
from src.utils import time_to_minutes
start_stop = "Jagodzińska"
stops_to_visit = ["most Grunwaldzki","Kochanowskiego","Wiśniowa"]
start_time = "12:00:00"


tabu_size = determine_tabu_size(len(stops_to_visit))
use_aspiration = True
max_iterations = 100
sample_size = 10
criterion = "time"  

tsp = TabuSearchTSP(
    stops=stops,
    criterion=criterion,
    transfer_time=3,
    tabu_size=tabu_size,
    use_aspiration=use_aspiration
)

best_solution = tsp.run(
    start_stop=start_stop,
    stops_to_visit=stops_to_visit,
    start_time=time_to_minutes(start_time),
    max_iterations=max_iterations,
    sample_size=sample_size
)

combined_route = tsp.combine_routes(best_solution)

print(format_compact_route(combined_route))
print(f"Najlepsza znaleziona trasa: {best_solution.stops_sequence}")




Tabu Search Progress:   3%|▎         | 3/100 [00:06<03:30,  2.17s/it]

🚉 SZCZEGÓŁY TRASY:
⏱️ Całkowity czas podróży: 92.0 min
🔄 Liczba przesiadek: 8

+-------+--------------------+--------------------+----------+----------+
| Linia | Skąd               | Dokąd              | Odjazd   | Przyjazd |
+-------+--------------------+--------------------+----------+----------+
| 110   | Jagodzińska        | Morwowa            | 12:00:00 | 12:05:00 |
| 136   | Morwowa            | Złotostocka        | 12:09:00 | 12:10:00 |
| 143   | Złotostocka        | 8 Maja             | 12:14:00 | 12:27:00 |
| 9     | 8 Maja             | Kochanowskiego     | 12:30:00 | 12:35:00 |
| 111   | Kochanowskiego     | PL. GRUNWALDZKI    | 12:36:00 | 12:40:00 |
| 145   | PL. GRUNWALDZKI    | DWORZEC AUTOBUSOWY | 12:44:00 | 12:58:00 |
| 15    | DWORZEC AUTOBUSOWY | Wiśniowa           | 13:03:00 | 13:09:00 |
| 136   | Wiśniowa           | Morwowa            | 13:14:00 | 13:24:00 |
| 145   | Morwowa            | Jagodzińska        | 13:27:00 | 13:32:00 |
+-------+--------------------+---

## Wizualizacja trasy

In [16]:
map_viz = visualize_route(combined_route, stops, "Trasa - Tabu Search")
display(map_viz)

## Wprowadzone modyfikacje

### 1. Dynamiczny dobór rozmiaru listy Tabu
Rozmiar listy Tabu ma kluczowe znaczenie dla efektywności algorytmu. Zbyt mała lista może prowadzić do cyklicznych powrotów do tych samych rozwiązań, podczas gdy zbyt duża może nadmiernie ograniczyć przestrzeń poszukiwań.

Wprowadzona modyfikacja polega na dynamicznym dostosowaniu rozmiaru listy Tabu w zależności od rozmiaru problemu:


```python
    def determine_tabu_size(num_stops):
    if num_stops <= 5:
        return max(3, num_stops // 2)
    elif num_stops <= 15:
        return max(5, num_stops // 2)
    else:
        return max(7, num_stops // 2)
```


Powyższa funkcja wykorzystuje heurystykę, według której rozmiar listy Tabu powinien być proporcjonalny do rozmiaru problemu, z pewnymi minimalnymi wartościami dla każdej kategorii wielkości problemu.

### 2. Zaawansowany mechanizm aspiracji

Standardowe kryterium aspiracji pozwala na wykonanie ruchu z listy tabu, jeśli prowadzi on do rozwiązania lepszego niż dotychczas najlepsze. Wprowadzona modyfikacja rozszerza ten mechanizm o:

1. **Historię wykorzystania ruchów:** Każdy wykonany ruch jest zapisywany w historii wraz z licznikiem jego wykorzystania.
2. **Mechanizm zaniku:** Wartości w historii są systematycznie zmniejszane (mnożone przez współczynnik zaniku), co preferuje ruchy, które nie były używane przez dłuższy czas.
3. **Kryterium częstotliwości:** Ruch z listy tabu może zostać wykonany, jeśli jest rzadko używany (jego wartość w historii jest poniżej progu).

```python
def _aspiration_criterion(self, new_cost, best_cost, move):
    if not self.use_aspiration:
        return False
    
    # Kryterium aspiracji 1: Lepsze rozwiązanie niż najlepsze znane
    if new_cost < best_cost:
        return True
    
    # Kryterium aspiracji 2: Rzadko używany ruch
    move_freq = self.aspiration_history.get(move, 0)
    if move_freq < 0.5:  # Parametr progu aspiracji
        return True
    
    return False
```

Ten wielopoziomowy mechanizm aspiracji pozwala na bardziej elastyczne przeszukiwanie przestrzeni rozwiązań i skuteczniejsze unikanie optymów lokalnych.

### 3. Inteligentne próbkowanie sąsiedztwa

Dla dużych problemów generowanie i ocena wszystkich możliwych sąsiadów może być bardzo kosztowne obliczeniowo. Wprowadzona strategia próbkowania:

1. **Parametryzacja rozmiaru próbki:** Użytkownik może określić maksymalną liczbę sąsiadów do wygenerowania i oceny.
2. **Losowe próbkowanie:** Zamiast generować wszystkie możliwe pary do zamiany, wybierana jest losowa podpróbka.

```python
if sample_size and len(swap_pairs) > sample_size:
    swap_pairs = random.sample(swap_pairs, sample_size)
```

Ta modyfikacja znacząco przyspiesza działanie algorytmu dla dużych problemów, zachowując jednocześnie zdolność do znajdowania dobrych rozwiązań.

# Wnioski i podsumowanie

Podczas realizacji zadań z listy udało mi się zaimplementować algorytmy wyszukiwania optymalnych ścieżek w grafie komunikacji miejskiej Wrocławia. 

Podczas implementacji zmagałem się z kilkoma problemami:

- Trudności z przetwarzaniem danych z pliku CSV (czas, duplikaty)
- Odpowiednie zdefiniowanie sąsiedztwa węzłów, uwzględniające specyfikę przesiadek
- Dobór funkcji heurystycznej dla A*, która dawałaby dobre wyniki bez utraty optymalności
- Znalezienie optymalnego rozmiaru listy Tabu - zbyt mała powodowała zapętlenia w rozwiązaniach, zbyt duża zwiększała złożoność obliczeniową

Realizacja zadań pozwoliła mi lepiej zrozumieć praktyczne aspekty algorytmów optymalizacyjnych i różnice między metodami dokładnymi a heurystycznymi.